<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Algorithmic_Head_and_Shoulders_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd

# Backtesting parameters (adjust as needed)
tickers = ['^GSPC']
periods = ['5y', '10y', '25y']
init_portfolio_value = 1000 # USD

for period in periods:
    for ticker in tickers:
        try:
            data = yf.download(ticker, period=period, progress=False)

            # Calculate indicators
            data['HL_avg'] = data['High'].rolling(window=25).mean() - data['Low'].rolling(window=25).mean()
            data['IBS'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'])
            data['Band'] = data['High'].rolling(window=25).mean() - (data['HL_avg'] * 2.25)
        except:
            continue

        # Initialize variables for trading strategy
        portfolio_value, portfolio_values = init_portfolio_value, [init_portfolio_value]
        trades = {'Entry': [], 'Exit': []}
        fees = 0
        position = False
        entry_price = 0

        # Simulate buying and selling using mentioned strategy
        for i in range(25, len(data)):
            if not position and data['Close'][i] < data['Band'][i] and data['IBS'][i] < 0.6: # Buy
                position = True
                entry_price = data['Close'][i]
                shares = portfolio_values[-1] / entry_price
                trades['Entry'].append(data.index[i])
            elif position and data['Close'][i] > data['High'][i - 1]: # Sell
                position = False
                exit_price = data['Close'][i]
                sale_value = shares * exit_price
                fee = max(.01, round((sale_value / 1000) * .01, 2))
                fees += fee
                trade_profit = shares * (exit_price - entry_price) - fee
                portfolio_value += trade_profit
                trades['Exit'].append(data.index[i])

            portfolio_values.append(portfolio_value)

        mean_reversal_return = portfolio_values[-1] / init_portfolio_value
        buy_and_hold_return = data['Close'].iloc[-1] / data['Close'].iloc[0]

        print(f'{ticker} made {mean_reversal_return*100:.1f}% return vs. {buy_and_hold_return*100:.1f}% in {period} time')

^GSPC made 157.7% return vs. 187.2% in 5y time
^GSPC made 226.8% return vs. 282.2% in 10y time
^GSPC made 471.3% return vs. 404.1% in 25y time


code offered in the article: https://medium.com/@minkeliu_29243/the-head-and-shoulders-pattern-in-technical-analysis-48cbca1ca9ea